# Тестирование и отладка

### Тестируем код


Время от времени я делаю небольшое изменение в своем коде и говорю себе: «Выглядит неплохо, можно отправлять». 

А затем все ломается. Ой! Каждый раз, когда
я делаю это (к счастью, со временем все реже и реже), я чувствую себя глупцом и клянусь, что в следующий раз напишу еще больше тестов.

Самый простой способ протестировать программы, написанные на Python, —добавить команды print(). 

Read-Evaluate-Print Loop (REPL) интерактивного интерпретатора позволяет вам быстро изменять код и тестировать изменения. 

Однако в производственном коде выражения print() использовать не стоит, поэтому вам нужно помнить о том, что следует удалять их. 

К тому же ошибки, связанные с копированием и вставкой, сделать очень легко.

### pylint, pyflakes и PEP-8

Следующим шагом перед созданием настоящих программ для тестирования является использование контролера кода Python. 

Самыми популярными являются pylint (http://www.pylint.org/) и pyflakes (http://bit.ly/pyflakes). 

Вы можете установить любой из них (или даже оба) с помощью pip:

$ pip install pylint
$ pip install pyflakes

Они проверяют на наличие реальных ошибок в коде (например, обращения к переменной до присвоения ей значения) и несоответствие стилю (как если бы
код носил одновременно одежду в полоску и в клетку). 

Рассмотрим практически бессмысленную программу, в которой есть логическая и стилистическая ошибки:

In [ ]:
a = 1
b = 2
print(a)
print(b)
print(c)

Так выглядит выходная информация от pylint:


 pylint style1.py

No config file found, using default configuration
************* Module style1
C: 1,0: Missing docstring
C: 1,0: Invalid name "a" for type constant
 (should match (([A-Z_][A-Z0-9_]*)|(__.*__))$)
C: 2,0: Invalid name "b" for type constant
 (should match (([A-Z_][A-Z0-9_]*)|(__.*__))$)
E: 5,6: Undefined variable 'c'

Если пролистать дальше, к разделу Global evaluation, можно увидеть наш счет
(10.0 — это высший балл)

Your code has been rated at -3.33/10

Ой! Сначала исправим ошибку. Строка вывода pylint, которая начинается с E, указывает на то, что найдена ошибка, которая заключается в том, что мы не присвоили значение переменной с до ее вывода на экран. 

Давайте это исправим:

In [ ]:
a = 1
b = 2
c = 3
print(a)
print(b)
print(c)
$ pylint style2.py

No config file found, using default configuration
************* Module style2
C: 1,0: Missing docstring
C: 1,0: Invalid name "a" for type constant
 (should match (([A-Z_][A-Z0-9_]*)|(__.*__))$)
C: 2,0: Invalid name "b" for type constant
(should match (([A-Z_][A-Z0-9_]*)|(__.*__))$)
C: 3,0: Invalid name "c" for type constant
 (should match (([A-Z_][A-Z0-9_]*)|(__.*__))$)

(should match (([A-Z_][A-Z0-9_]*)|(__.*__))$)
Отлично, больше строк, начинающихся с Е, нет. Наш счет увеличился с -3.33
до 4.29:

Your code has been rated at 4.29/10

pylint хочет увидеть строку документации (короткий текстовый фрагмент в верхней части модуля или функции, описывающий код) и считает, что короткие имена переменных вроде a, b и c не очень аккуратны. 

Сделаем pylint счастливее, а наш код — еще лучше:

In [ ]:
"Module docstring goes here"
def func():
    "Function docstring goes here. Hi, Mom!"
    first = 1
    second = 2
    third = 3
    print(first)
    print(second)
    print(third)
func()

In [ ]:
$ pylint style3.py
No config file found, using default configuration

Жалоб нет. А какой у нас счет?

Your code has been rated at 10.00/10

Не так уж и плохо?
Еще одним контролером стиля является PEP-8 (https://pypi.python.org/pypi/pep8),
вы можете установить его привычным способом:

$ pip install pep8

Что он скажет о нашей последней версии кода?

$ pep8 style3.py

style3.py:3:1: E302 expected 2 blank lines, found 1

Чтобы сделать код еще более стильным, он рекомендует добавить пустую строку после начальной строки документации модуля.

### unittest

Мы убедились, что больше не оскорбляем чувство стиля богов кода, поэтому теперь можно перейти к настоящим тестам логики вашей программы.

Хорошим тоном является написание и запуск тестовых программ до отправки кода в систему контроля исходного кода. 

Написание тестов поначалу может быть утомительным, но они действительно помогают вам находить проблемы быстрее,
особенно регрессионные тесты (суть которых заключается в том, чтобы сломать то, что раньше работало). 

Болезненный опыт учит всех разработчиков тому, что
даже самое маленькое изменение, которое, по их заверениям, не затрагивает другие области приложения, на самом деле влияет на них. 

Если вы взглянете на качественные пакеты Python, то заметите, что они поставляются с набором тестов.

Стандартная библиотека содержит не один, а целых два пакета для тестирования приложений. 

Начнем с unittest (https://docs.python.org/3/library/unittest.html). 

Мы напишем модуль, который записывает слова с прописной буквы. 

Наша первая версия будет использовать стандартную строковую функцию capitalize(), что, как вы увидите, приведет к неожиданным результатам. 

Сохраните этот файл под именем cap.py:

In [ ]:
def just_do_it(text):
    return text.capitalize()

Основная идея тестирования заключается в том, чтобы понять, какой результат вы хотите получить при определенных входных данных (в нашем примере вы
хотите получить текст, который ввели, записанным с прописной буквы), отправить результат функции тестирования, а затем проверить, получен ли ожидаемый
результат. 

Ожидаемый результат называется утверждением (assertion), поэтому в рамках пакета unittest вы проверяете результат с помощью методов, чьи имена начинаются со слова assert, например assertEqual, показанного в следующем
примере.

Сохраните этот сценарий тестирования под именем test_cap.py:

In [ ]:
import unittest
import cap
class TestCap(unittest.TestCase):
    def setUp(self):
        pass
    def tearDown(self):
        pass
    def test_one_word(self):
        text = 'duck'
        result = cap.just_do_it(text)
        self.assertEqual(result, 'Duck')
    def test_multiple_words(self):
        text = 'a veritable flock of ducks'
        result = cap.just_do_it(text)
        self.assertEqual(result, 'A Veritable Flock Of Ducks')
if __name__ == '__main__':
    unittest.main()

Перед каждым методом тестирования вызывается метод setUp(), а после каждого из методов тестирования — метод tearDown(). 

Задачей этих методов является выделение и освобождение внешних ресурсов, необходимых для тестов, вроде соединения с базой данных или создания некоторых тестовых данных. 

В нашем случае тесты автономны, и нам даже ненужно определять методы setUp() и tearDown(), однако создать их пустые версии не повредит. 

Сердцем наших тестов являются две функции с именами test_one_word() и test_multiple_words(). 

Каждая из них запускает определенную нами функцию just_do_it() с разными входными параметрами и проверяет, получен ли ожидаемый результат.

О’кей, запустим тест. Эта команда вызовет два наших метода тестирования:

$ python test_cap.py

In [ ]:
F.
 ======================================================================
 FAIL: test_multiple_words (__main__.TestCap)
 ----------------------------------------------------------------------
 Traceback (most recent call last):
 File "test_cap.py", line 20, in test_multiple_words
 self.assertEqual(result, 'A Veritable Flock Of Ducks')
 AssertionError: 'A veritable flock of ducks' != 'A Veritable Flock Of Ducks'
— A veritable flock of ducks
 ? ^ ^ ^ ^
 + A Veritable Flock Of Ducks
 ? ^ ^ ^ ^
 ----------------------------------------------------------------------
Ran 2 tests in 0.001s
 FAILED (failures=1)

Пакет устроил результат первой проверки (test_one_word), но не результат второй (test_multiple_words). 

Стрелки вверх (^) показывают, какие строки отличаются.

Что такого особенного в примере с несколькими словами? 

После прочтения документации для строковой функции capitalize (https://docs.python.org/3/library/
stdtypes.html#str.capitalize) мы поняли причину проблемы: она увеличивает только первую букву первого слова. 

Возможно, нам сразу нужно было начать с чтения
документации.

Нам нужна другая функция. После прочтения той страницы мы нашли функцию
title( (https://docs.python.org/3/library/stdtypes.html#str.title). 

Изменим файл cap.py так, чтобы в нем вместо функции capitalize() использовалась функция title():

Повторите тесты и взгляните на результат:

$ python test_cap.py
 ..
 ----------------------------------------------------------------------
Ran 2 tests in 0.000s
OK

Все прошло отлично. Хотя на самом деле нет. Нам нужно добавить в файл
test_cap.py как минимум еще один метод:


In [ ]:
def test_words_with_apostrophes(self):
    text = "I'm fresh out of ideas"
    result = cap.just_do_it(text)
    self.assertEqual(result, "I'm Fresh Out Of Ideas")


Запустите тесты еще раз:

In [ ]:
$ python test_cap.py
..F
 ======================================================================
 FAIL: test_words_with_apostrophes (__main__.TestCap)
 ----------------------------------------------------------------------
 Traceback (most recent call last):
 File "test_cap.py", line 25, in test_words_with_apostrophes
 self.assertEqual(result, "I'm Fresh Out Of Ideas")
 AssertionError: "I'M Fresh Out Of Ideas" != "I'm Fresh Out Of Ideas"
— I'M Fresh Out Of Ideas
 ? ^
 + I'm Fresh Out Of Ideas
 ? ^
 ----------------------------------------------------------------------
Ran 3 tests in 0.001s
 FAILED (failures=1)

Наша функция увеличила букву m в конструкции I'm. 

В документации к функции title() мы обнаружили, что она плохо работает с апострофами. 

Нам действительно стоило сначала прочитать ее текст целиком.

В самом конце документации стандартной библиотеки, касающейся строк, мы находим еще одного кандидата — вспомогательную функцию с именем capwords().

Используем ее в файле cap.py:

In [ ]:
def just_do_it(text):
    from string import capwords
    return capwords(text)

$ python test_cap.py

In [ ]:
 ...
 ----------------------------------------------------------------------
Ran 3 tests in 0.004s
OK

Наконец-то мы это сделали! Э-э-э, на самом деле нет. Нужно добавить еще один тест в файл test_cap.py:

In [ ]:
def test_words_with_quotes(self):
    text = "\"You're despicable,\" said Daffy Duck"
    result = cap.just_do_it(text)
    self.assertEqual(result, "\"You're Despicable,\" Said Daffy Duck")

Сработало?

In [ ]:
$ python test_cap.py
 ...F
 ======================================================================
 FAIL: test_words_with_quotes (__main__.TestCap)
 ----------------------------------------------------------------------
 Traceback (most recent call last):
 File "test_cap.py", line 30, in test_words_with_quotes
 self.assertEqual(result, "\"You're
 Despicable,\" Said Daffy Duck")
 AssertionError: '"you\'re Despicable," Said Daffy Duck'
 != '"You\'re Despicable," Said Daffy Duck'
— "you're Despicable," Said Daffy Duck
 ? ^
 + "You're Despicable," Said Daffy Duck
 ? ^
 ----------------------------------------------------------------------
Ran 4 tests in 0.004s
 FAILED (failures=1)

Выглядит так, будто первая двойная кавычка смутила даже функцию capwords, нашего текущего фаворита. 

Она попробовала увеличить символ " и уменьшить все
остальное (You're). Нам также нужно проверить, оставила ли функция-увеличитель остальную часть строки нетронутой.

Люди, которые зарабатывают тестированием на жизнь, способны замечать такие крайние случаи, но разработчики часто забывают о таких ситуациях, когда речь
идет об их собственном коде.

Пакет unittest предоставляет небольшой, но мощный набор операторов, позволяющих вам проверять значения, убеждаться в том, что вы имеете необходимый
класс, определять, сгенерировалась ли ошибка, и т. д.

### Пакет doctest

Вторым пакетом для тестирования стандартной библиотеки является doctest (http://bit.ly/py-doctest). 

С помощью этого пакета вы можете писать тесты внутри
строки документации, которые и сами будут служить документацией. 

Он выглядит как интерактивный интерпретатор: символы >>>, за которыми следует вызов, а затем результаты в следующей строке. 

Вы можете запустить некоторые тесты в интерактивном интерпретаторе и просто вставить результат в свой тестовый файл.

Мы модифицируем файл cap.py (убрав тот проблемный тест с кавычками):

In [ ]:
def just_do_it(text):
    """
    >>> just_do_it('duck')
    'Duck'
    >>> just_do_it('a veritable flock of ducks')
    'A Veritable Flock Of Ducks'
    >>> just_do_it("I'm fresh out of ideas")
    "I'm Fresh Out Of Ideas"
    """
    from string import capwords
    return capwords(text)
if __name__ == '__main__':
    import doctest
    doctest.testmod()


Когда вы его запустите, в случае успеха он не выведет ничего:

$ python cap.py

Запустите его с опцией -v (verbose, verbose), чтобы увидеть, что произошло на
самом деле:

$ python cap.py -v

 Trying:
 just_do_it('duck')
 Expecting:
 'Duck'
 ok
 Trying:
 just_do_it('a veritable flock of ducks')
 Expecting:
 'A Veritable Flock Of Ducks'
 ok
 Trying:
 just_do_it("I'm fresh out of ideas")
 Expecting:
 "I'm Fresh Out Of Ideas"
 ok
 1 items had no tests:
 __main__
 1 items passed all tests:
 3 tests in __main__.just_do_it
 3 tests in 2 items.
3 passed and 0 failed.
Test passed.

### Пакет nose

https://github.com/nose-devs/nose2

Сторонний пакет nose (https://nose.readthedocs.org/en/latest/) — это еще одна альтернатива пакету unittest. Команда, позволяющая установить его, выглядит так:

$ pip install nose

Вам не нужно создавать класс, который содержит тестовые методы, как мы делали при работе с unittest. 

Любая функция, содержащая в своем имени слово
test, будет запущена. 

Модифицируем нашего последнего тестировщика Unittest
и сохраним его под именем test_cap_nose.py:

In [ ]:
import cap
from nose.tools import eq_
def test_one_word():
    text = 'duck'
    result = cap.just_do_it(text)
    eq_(result, 'Duck')
def test_multiple_words():
    text = 'a veritable flock of ducks'
    result = cap.just_do_it(text)
    eq_(result, 'A Veritable Flock Of Ducks')
def test_words_with_apostrophes():
    text = "I'm fresh out of ideas"
    result = cap.just_do_it(text)
    eq_(result, "I'm Fresh Out Of Ideas")
def test_words_with_quotes():
    text = "\"You're despicable,\" said Daffy Duck"
    result = cap.just_do_it(text)
    eq_(result, "\"You're Despicable,\" Said Daffy Duck")


Запустим тесты:

In [ ]:
$ nosetests test_cap_nose.py
 ...F
 ======================================================================
 FAIL: test_cap_nose.test_words_with_quotes
 ----------------------------------------------------------------------
 Traceback (most recent call last):
 File "/Users/.../site-packages/nose/case.py", line 198, in runTest
 self.test(*self.arg)
 File "/Users/.../book/test_cap_nose.py", line 23, in test_words_with_quotes
 eq_(result, "\"You're Despicable,\" Said Daffy Duck")
 AssertionError: '"you\'re Despicable," Said Daffy Duck'
 != '"You\'re Despicable," Said Daffy Duck'
----------------------------------------------------------------------
Ran 4 tests in 0.005s
 FAILED (failures=1)

Мы нашли ту же ошибку, что и при использовании unittest, к счастью, в конце этой главы приводится упражнение, в котором вам предстоит ее исправить.

### Другие фреймворки для тестирования

По некоторой причине людям нравится писать тестовые фреймворки для Python.

Если вам любопытно, можете взглянуть на другие популярные решения вроде tox и py.test.

### Постоянная интеграция

Когда ваша группа генерирует много кода каждый день, полезно автоматизировать тесты по мере появления изменений. 

Вы можете автоматизировать системы контроля версий так, чтобы тесты запускались по мере появления нового кода.

Таким образом, каждый будет знать, что кто-то сломал билд и убежал обедать пораньше.

Эти системы велики, и я не буду рассматривать детали их установки и использования. 

Если они вам когда-нибудь понадобятся, вы будете знать, где их искать.

 buildbot (http://buildbot.net/). Эта система контроля версий, написанная на Python, автоматизирует построение, тестирование и выпуск кода.

 jenkins (http://jenkins-ci.org/). Система написана на Java, она выглядит наиболее предпочтительным инструментом для постоянной интеграции в данный момент.

 travis-ci (http://travis-ci.com/). Эта система автоматизирует проекты, размещенные на GitHub, она бесплатна для проектов с открытым исходным кодом.

# Отлаживаем свой код

Всегда тестируйте свой код. Чем лучше тесты, тем меньше вам предстоит работы в дальнейшем. 

Однако ошибки случаются, и их нужно исправлять. Самый простой способ выполнять отладку в Python — построчно выполнять код. 

Полезно отображать результат работы функции vars(), которая извлекает значения ваших локальных переменных, включая аргументы функций:

https://pythonz.net/references/named/vars/

In [1]:
def func(*args, **kwargs):
    print(vars())
func(1, 2, 3)

{'args': (1, 2, 3), 'kwargs': {}}


In [2]:
func(['a', 'b', 'argh'])

{'args': (['a', 'b', 'argh'],), 'kwargs': {}}


Как вы уже знаете из раздела «Декораторы» главы 4, декоратор может вызывать код, располагающийся до или после функции, не модифицируя код внутри самой
функции. 

Это значит, что вы можете использовать декоратор, чтобы выполнить какое-либо действие до или после вызова любой функции, а не только тех, которые написали вы. 

Определим декоратор dump, который позволяет вывести на экран входные аргументы и выводимые значения любой функции по мере ее вызова (дизайнеры знают, что выходные данные нужно декорировать):

In [3]:
def dump(func):
    "Print input arguments and output value(s)"
    def wrapped(*args, **kwargs):
        print("Function name: %s" % func.__name__)
        print("Input arguments: %s" % ' '.join(map(str, args)))
        print("Input keyword arguments: %s" % kwargs.items())
        output = func(*args, **kwargs)
        print("Output:", output)
        return output
    return wrapped

Перейдем к декорируемой части. Это функция с именем double(), которая принимает именованные или безымянные числовые аргументы и возвращает их удвоенные значения в списке:

In [4]:
@dump
def double(*args, **kwargs):
    "Double every argument"
    output_list = [ 2 * arg for arg in args ]
    output_dict = { k:2*v for k,v in kwargs.items() }
    return output_list, output_dict

output = double(3, 5, first=100, next=98.6, last=-40)


Function name: double
Input arguments: 3 5
Input keyword arguments: dict_items([('first', 100), ('next', 98.6), ('last', -40)])
Output: ([6, 10], {'first': 200, 'next': 197.2, 'last': -80})


# Отлаживаем с помощью pdb

Эти приемы полезны, но иногда ничто не сможет заменить настоящий отладчик.

Большинство IDE содержат отладчики, чьи возможности и пользовательские интерфейсы могут варьироваться. 

В этом разделе я опишу использование стандартного отладчика Python pdb (https://docs.python.org/3/library/pdb.html).

Рассмотрим программу с ошибкой, которая зависит от входных данных, — такую ошибку может быть особенно трудно найти. 

Это реальная ошибка, возникшая в ранние дни программирования, она довольно долго сбивала с толку
программистов.

Мы собираемся считать файл, содержащий названия стран и их столиц, разделенные запятыми, и вывести их на экран в формате «столица, страна». 

Учтите, что прописные буквы в словах могут быть неправильно расставлены, поэтому нам нужно исправить это при выводе на экран. 

В файле также могут быть лишние пробелы, вам нужно избавиться и от них. 

Наконец, несмотря на то что было бы логично считать весь файл до конца, по какой-то причине наш менеджер сказал нам остановиться, если мы встретим слово quit (состоящее из смеси прописных и строчных букв). 

Так выглядит файл с данными:

France, Paris
venuzuela,caracas
 LithuniA,vilnius
 quit

Разработаем алгоритм (способ решения задачи). 

Это псевдокод, он выглядит как программа, но является лишь способом выразить логику простым языком до преобразования его в настоящую программу. 

Одна из причин, по которым программисты любят Python, — он выглядит очень похожим на псевдокод, поэтому
его не так трудно преобразовать в рабочую программу:

для каждой строки в текстовом файле
считать строку
удалить пробелы в начале и конце строки
если найдена строка "quit" в строке, записанной в нижнем регистре
остановиться
иначе
разделить страну и столицу символом запятой
удалить пробелы в начале и конце
записать страну и столицу с прописной буквы
вывести на экран столицу, запятую и страну

Нам нужно удалить из имен начальные и конечные пробелы, поскольку это является требованием к программе. Аналогично мы поступаем со сравнением со строкой quit и записью названий страны и города с прописной буквы. 

Имея это в виду, напишем файл capitals.py, который точно будет работать корректно:

In [5]:
def process_cities(filename):
    with open(filename, 'rt') as file:
        for line in file:
            line = line.strip()
            if 'quit' in line.lower():
                return
            country, city = line.split(',')
            city = city.strip()
            country = country.strip()
            print(city.title(), country.title(), sep=',')

import sys
process_cities('read_this1.txt')

Paris,France
Caracas,Venuzuela
Vilnius,Lithunia


Выглядит отлично! Программа прошла один тест, поэтому отправим ее на производство обрабатывать столицы и страны со всего мира, пока она не ошибется на данном файле:

Программа завершается после вывода всего пяти строк, несмотря на то что
в файле их было 15, как показано здесь:

python capitals.py read_this2.txt

Buenos Aires,Argentina
La Paz,Bolivia
Brazilia,Brazil
Santiago,Chile
Bogotá,Colombia

Программа завершается после вывода всего пяти строк, несмотря на то что в файле их было 15, как показано здесь:

Что случилось? Мы можем продолжать редактировать файл capitals.py, размещая выражения print() в тех местах, где может возникнуть ошибка, но посмотрим, сможет ли нам помочь отладчик.

Для того чтобы использовать отладчик, импортируйте модуль pdb из командной строки, введя –m pdb, например, так:

In [ ]:
$ python -m pdb capitals.py read_this2.txt

> /Users/williamlubanovic/book/capitals.py(1)<module>()
-> def process_cities(filename):
(Pdb)

Это запустит программу и разместит вас на первой строке. Если вы введете символ с (от слова continue — «продолжить»), программа будет работать, пока не завершится либо естественным образом, либо из-за ошибки:

(Pdb) c
Buenos Aires,Argentina
La Paz,Bolivia
Brazilia,Brazil
Santiago,Chile
Bogotá,Colombia
The program finished and will be restarted
> /Users/williamlubanovic/book/capitals.py(1)<module>()
-> def process_cities(filename):


Программа завершилась нормально, точно так же, как и раньше, когда мы запускали ее вне отладчика. 

Попробуем запустить ее снова, использовав специальные
команды, чтобы сузить место поиска проблемы. 

Похоже, имеет место логическая ошибка, а не синтаксическая проблема или исключение (из-за них мы бы увидели сообщение об ошибке).

Введите s (step — «шаг»), чтобы пройти по отдельным строкам кода. 

Это позволит пройти по всем строкам — вашим, стандартной библиотеки и любых других используемых вами модулей. 

Когда вы применяете команду s, вы также входите
во все функции и проходите каждую построчно. 

Введите n (next — «следующий»), чтобы идти по шагам, но не заходить внутрь функций: когда вы находитесь на
строке, где вызывается функция, эта команда выполняет всю функцию и вы оказываетесь на следующей строке. 

Используйте s, если вы не уверены в том, где
конкретно есть проблема, а n — если уверены, что некоторая функция проблем не вызывает, особенно если это длинная функция. 

Зачастую вы будете проходить построчно весь свой код и пропускать библиотечный, поскольку подразумевается,
что он хорошо протестирован. 

Мы используем s, чтобы начать двигаться от начала
программы к функции process_cities():

 > /Users/williamlubanovic/book/capitals.py(12)<module>()
 -> if __name__ == '__main__':
(Pdb) s
 > /Users/williamlubanovic/book/capitals.py(13)<module>()
 -> import sys
(Pdb) s
 > /Users/williamlubanovic/book/capitals.py(14)<module>()
 -> process_cities(sys.argv[1])
(Pdb) s
 --Call--
 > /Users/williamlubanovic/book/capitals.py(1)process_cities()
 -> def process_cities(filename):
 (Pdb) s
 > /Users/williamlubanovic/book/capitals.py(2)process_cities()
 -> with open(filename, 'rt') as file:

Введите l (list — «перечислить»), чтобы увидеть следующие несколько строк
своей программы

(Pdb) l
 1 def process_cities(filename):
 2 -> with open(filename, 'rt') as file:
 3 for line in file:
 4 line = line.strip()
 5 if 'quit' in line.lower():
 6 return
 7 country, city = line.split(',')
 8 city = city.strip()
 9 country = country.strip()
10 print(city.title(), country.title(), sep=',')
11
(Pdb)

Стрелка (->) указывает на текущую строку.

Мы могли бы и дальше применять команды s или n в надежде что-то найти, но давайте использовать одну из главных особенностей отладчика — точки останова.

Точка останова останавливает выполнение программы на указанной вами строке.

В данном случае мы хотим узнать, почему функция process_cities() вызывает завершение программы до прочтения всех введенных строк. 

Строка 3 (for line in file:) будет считывать каждую строку входного файла, поэтому она выглядит невинно. 

Единственное место, где мы можем вернуться из функции до прочтения всех данных, — это строка 6 (return). 

Поставим точку останова на строке 6:

(Pdb) b 6

Breakpoint 1 at /Users/williamlubanovic/book/capitals.py:6


Далее продолжим выполнение программы до тех пор, пока она либо не достигнет точки останова, либо не завершится обычным образом:

(Pdb) c

 Buenos Aires,Argentina
 La Paz,Bolivia
 Brasilia,Brazil
 Santiago,Chile
 Bogotá,Colombia
> /Users/williamlubanovic/book/capitals.py(6)process_cities()
-> return


Ага, она остановилась на точке останова в строке 6. Это показывает, что программа хочет завершиться после прочтения страны, которая идет вслед за Колумбией. Выведем значение переменной line, чтобы увидеть, что мы только что считали:

(Pdb) p line

'ecuador,quito'

Что такого особенного… а, забудьте.
Серьезно? Столица называется *quit*o? Наш менеджер не ожидал, что строка
quit станет частью входных данных, поэтому показалось логичным использовать
ее в качестве контрольного значения (индикатора конца). Вам следует отправиться прямо к нему и сказать все как на духу, я подожду.
Если после этого у вас все еще есть работа, можете просмотреть все точки останова с помощью команды b:

(Pdb) b

Num Type Disp Enb Where
1 breakpoint keep yes at /Users/williamlubanovic/book/capitals.py:6
 breakpoint already hit 1 time

Команда l покажет вам строки кода, текущую строку (->) и все имеющиеся точки останова (B). 

Вызов команды l без аргументов выведет все строки, начиная с точки предыдущего вызова команды l, поэтому включите в вызов опциональный параметр — стартовую строку (в нашем примере начнем с 1):

(Pdb) l 1

 1 def process_cities(filename):
 2 with open(filename, 'rt') as file:
 3 for line in file:
 4 line = line.strip()
 5 if 'quit' in line.lower():
 6 B-> return
 7 country, city = line.split(',')
 8 city = city.strip()
 9 country = country.strip()
10 print(city.title(), country.title(), sep=',')

Теперь модифицируем наш тест так, чтобы выполнялась проверка на полное совпадение со строкой quit, без всяких других символов:

In [ ]:
def process_cities(filename):
    with open(filename, 'rt') as file:
        for line in file:
            line = line.strip()
            if 'quit' == line.lower():
                return
            country, city = line.split(',')
            city = city.strip()
            country = country.strip()
            print(city.title(), country.title(), sep=',')

import sys
process_cities(sys.argv[1])

Запустим программу еще раз:

In [ ]:
$ python capitals2.py cities2.csv

In [ ]:
Buenos Aires,Argentina
La Paz,Bolivia
Brasilia,Brazil
Santiago,Chile
Bogotá,Colombia
Quito,Ecuador
Stanley,Falkland Islands
Cayenne,French Guiana
Georgetown,Guyana
Asunción,Paraguay
Lima,Peru
Paramaribo,Suriname
Montevideo,Uruguay
Caracas,Venezuela

Только что мы кратко рассмотрели отладчик — этого достаточно, чтобы показать
вам, что вы можете сделать и какие команды будете использовать большую часть
времени.
Помните: больше тестов — меньше отладки.

# Записываем в журнал сообщения об ошибках

В какой-то момент вам может понадобиться перейти от использования выражений print() к записи сообщений в журнал. 

Журнал, как правило, представляет собой системный файл, в котором накапливаются сообщения, содержащие полезную информацию вроде временной метки или имени пользователя, запустившего программу. 

Зачастую журналы ежедневно ротируются (переименовываются) и сжимаются, благодаря чему они не переполняют ваш диск и не создают проблем. 

Если у вашей программы что-то пошло не так, вы можете просмотреть соответствующий файл журнала, чтобы увидеть, что произошло. 

Содержимое исключений особенно полезно записывать в журнал, поскольку оно подсказывает вам номер строки, после выполнения которой программа завершилась, и причину
такого завершения.

Для журналирования используется модуль стандартной библиотеки logging (http://bit.ly/py-logging). 

Большинство его описаний я считаю немного непонятными.

Спустя некоторое время эти описания будут казаться осмысленными, но в первый раз они выглядят чересчур сложными. 

Модуль logging содержит следующие концепции:

 сообщение, которое вы хотите сохранить в журнал;

 уровни приоритета и соответствующие функции — debug(), info(), warn(), error() и critical();

 один или несколько объектов журналирования для основной связи с модулем;

 обработчики, которые направляют значение в терминал, файл, базу данных или куда-нибудь еще;

 средства форматирования выходных данных;

 фильтры, которые принимают решения в зависимости от входных данных.

Рассмотрим простейший пример журналирования — просто импортируем модуль и воспользуемся некоторыми из его функций:

In [7]:
import logging
logging.debug("Looks like rain")
logging.info("And hail")
logging.warn("Did I hear thunder?")


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  after removing the cwd from sys.path.


In [8]:
logging.error("Was that lightning?")

ERROR:root:Was that lightning?


In [9]:
logging.critical("Stop fencing and get inside!")


CRITICAL:root:Stop fencing and get inside!


Вы заметили, что вызовы debug() и info() не сделали ничего, а два других вывели на экран строку УРОВЕНЬ:root:перед каждым сообщением? 

Пока они выглядят как выражение print(), имеющее несколько личностей.

Но это полезно. Вы можете выполнить поиск определенного значения уровня в журнале, чтобы найти определенные сообщения, сравнить временные метки и увидеть, что случилось перед тем, как упал ваш сервер, и т. д.

Усиленные раскопки в документации отвечают на первую загадку (на вторую мы ответим уже через пару страниц): 

уровень приоритета по умолчанию — WARNING,
он будет записан в журнал, когда мы вызовем первую функцию (logging.debug()).

Мы можем указать уровень по умолчанию с помощью функции basicConfig().

Самый низкий уровень — DEBUG, это дает возможность поймать более высокие уровни:

In [11]:
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug("It's raining again")

DEBUG:root:It's raining again

In [12]:
logging.info("With hail the size of hailstones")

INFO:root:With hail the size of hailstones

Мы сделали это с помощью стандартных функций журналирования, не создавая специализированный объект.

Каждый объект журналирования имеет имя. 

Создадим объект, который называется bunyan:

In [ ]:
import logging
logging.basicConfig(level='DEBUG')
logger = logging.getLogger('bunyan')
logger.debug('Timber!')

DEBUG:bunyan:Timber!

Если имя объекта журналирования содержит точки, они разделяют уровни иерархии таких объектов, каждый из которых потенциально имеет разные приоритеты. 

Это означает, что объект с именем quark выше, чем объект с именем quark.charmed. 

На вершине иерархии находится корневой объект журналирования с именем ' '.

До этого момента мы только выводили сообщения, что практически не отличается от функции print().

Чтобы направить сообщения в разные места назначения,
используем обработчики. 

Самым распространенным местом назначения является
файл журнала, направить туда сообщения можно так:

In [13]:
import logging
logging.basicConfig(level='DEBUG', filename='blue_ox.log')
logger = logging.getLogger('bunyan')
logger.debug("Where's my axe?")
logger.warn("I need my axe")


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  """


In [ ]:
Ага, строки больше не показываются на экране, вместо этого они попадают
в файл blue_ox.log:

DEBUG:bunyan:Where's my axe?
WARNING:bunyan:I need my axe

Вызов функции basicConfig() и передача имени файла в качестве аргумента создали для вас объект типа FileHandler и сделали его доступным объекту журналирования. 

Модуль журналирования содержит как минимум 15 обработчиков для отправки сообщений в разные места вроде электронной почты, веб-серверов, экранов и файлов.

Наконец, вы можете управлять форматом сообщений журнала.

В нашем первом примере мы использовали формат, применяемый по умолчанию, в результате чего
появилась следующая строка:

WARNING:root:Message..

Если вы предоставите строку format функции basicConfig(), то можете изменить формат по собственному желанию:

In [14]:
import logging
fmt = '%(asctime)s %(levelname)s %(lineno)s %(message)s'
logging.basicConfig(level='DEBUG', format=fmt)
logger = logging.getLogger('bunyan')
logger.error("Where's my other plaid shirt?")

ERROR:bunyan:Where's my other plaid shirt?


2014-04-08 23:13:59,899 ERROR 1 Where's my other plaid shirt?

Мы позволили объекту журналирования снова отправить выходные данные на экран, но изменили их формат. 

Модуль logging распознал количество имен переменных в строке формата fmt. 

Мы использовали asctime (дата и время как строка 
ISO 8601), levelname, lineno (номер строки) и само сообщение в переменной message.

Существуют и другие встроенные переменные, вы можете предоставить и свои собственные переменные.

Пакет logging содержит гораздо больше особенностей, чем можно описать в этом небольшом обзоре. 

Вы можете писать в несколько журналов одновременно, указывая разные приоритеты и форматы. Этот пакет довольно гибок, но иногда это достигается за счет простоты.